# Appendix
### Individual discipline cleaning
Some cleaning of the individual discipline datasets was needed to get all the datasets in the right place before concatenating into one in the main notebook. Kept in separate notebook to avoid bloating the main analysis.

In [84]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [85]:
%matplotlib inline

In [138]:
# Read the .csvs into a pandas DataFrame for each discipline
ws = pd.read_csv('data_original/ws.csv')
wd = pd.read_csv('data_original/wd.csv')
md = pd.read_csv('data_original/md.csv')
xd = pd.read_csv('data_original/xd.csv')

#### Initial pre-processing

In [139]:
wd['team_one_players'] = wd['team_one_player_one'] + ', ' + wd['team_one_player_two']
wd['team_two_players'] = wd['team_two_player_one'] + ', ' + wd['team_two_player_two']
md['team_one_players'] = md['team_one_player_one'] + ', ' + md['team_one_player_two']
md['team_two_players'] = md['team_two_player_one'] + ', ' + md['team_two_player_two']
xd['team_one_players'] = xd['team_one_player_one'] + ', ' + xd['team_one_player_two']
xd['team_two_players'] = xd['team_two_player_one'] + ', ' + xd['team_two_player_two']

In [140]:
team_one = wd.pop('team_one_players')
team_two = wd.pop('team_two_players')
wd.insert(13, 'team_one_players', team_one)
wd.insert(14, 'team_two_players', team_two)

In [141]:
team_one = md.pop('team_one_players')
team_two = md.pop('team_two_players')
md.insert(13, 'team_one_players', team_one)
md.insert(14, 'team_two_players', team_two)

In [142]:
team_one = xd.pop('team_one_players')
team_two = xd.pop('team_two_players')
xd.insert(13, 'team_one_players', team_one)
xd.insert(14, 'team_two_players', team_two)

#### Women's Doubles (WD)

In [143]:
wd['game_2_score'] = wd['game_2_score'].fillna('0-0')
wd['game_3_score'] = wd['game_3_score'].fillna('0-0')
wd['team_one_most_consecutive_points_game_2'] = wd['team_one_most_consecutive_points_game_2'].fillna(0)
wd['team_two_most_consecutive_points_game_2'] = wd['team_two_most_consecutive_points_game_2'].fillna(0)
wd['team_one_most_consecutive_points_game_3'] = wd['team_one_most_consecutive_points_game_3'].fillna(0)
wd['team_two_most_consecutive_points_game_3'] = wd['team_two_most_consecutive_points_game_3'].fillna(0)
wd['team_one_game_points_game_2'] = wd['team_one_game_points_game_2'].fillna(0)
wd['team_two_game_points_game_2'] = wd['team_two_game_points_game_2'].fillna(0)
wd['team_one_game_points_game_3'] = wd['team_one_game_points_game_3'].fillna(0)
wd['team_two_game_points_game_3'] = wd['team_two_game_points_game_3'].fillna(0)

In [144]:
wd = wd[wd.retired ==  False]
wd.drop(columns=['retired'], inplace=True)

In [145]:
# Create two new columns containing each player's points in game one by splitting 'game_1_score'
wd[['team_one_game_1_points', 'team_two_game_1_points']] = wd.game_1_score.str.split('-', expand=True)
wd[['team_one_game_2_points', 'team_two_game_2_points']] = wd.game_2_score.str.split('-', expand=True)
wd[['team_one_game_3_points', 'team_two_game_3_points']] = wd.game_3_score.str.split('-', expand=True)

In [146]:
# Replace Null values, which we identified do not occur in any game one columns, with zero
wd['team_one_game_2_points'] = wd['team_one_game_2_points'].fillna(0)
wd['team_two_game_2_points'] = wd['team_two_game_2_points'].fillna(0)
wd['team_one_game_3_points'] = wd['team_one_game_3_points'].fillna(0)
wd['team_two_game_3_points'] = wd['team_two_game_3_points'].fillna(0)

In [147]:
# Create new columns based on above with number representing player who won each game
# As game can go beyond 21 condition is simply >
# np.where() returns player one ('1') if condition is met, else returns player two ('2')
wd['game_1_winner'] = np.where(wd['team_one_game_1_points'] > wd['team_two_game_1_points'], 1, 2)
wd['game_2_winner'] = np.where(wd['team_one_game_2_points'] > wd['team_two_game_2_points'], 1, 2)
wd['game_3_winner'] = np.where(wd['team_one_game_3_points'] > wd['team_two_game_3_points'], 1, 2)

In [148]:
# Replace the erroneous '2's with '0's when a game three was not played
wd.loc[((wd.team_one_game_3_points==0) & (wd.team_two_game_3_points==0)), 'game_3_winner']=0

In [149]:
# Function to find the first 'instance' of 11 points being scored, signalling the interval
def interval_score(array):
    '''
    extract score on interval (11 points) on a single game
    '''
    
    for a in array:
        if '11' in a:
            interval_score = a.strip("'] [ ")
            return interval_score
            break

In [150]:
# Extract the score at the mid-game interval from the List of each games' score.
wd['game_1_interval_score'] = wd['game_1_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')
wd['game_2_interval_score'] = wd['game_2_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')
wd['game_3_interval_score'] = wd['game_3_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')

In [151]:
# Function to identify which player reached the interal by regex searching the scoreline string for Player 1 reaching 11, else Player 2
def interval_winner(string):
    if re.search('^11', string):
        player = 1
    else:
        player = 2
    return player

In [152]:
# Apply above function to dataframe, we can now compare how influential reaching the Game 1 interval is to winning the first game
wd['game_1_interval_winner'] = wd['game_1_interval_score'].apply(lambda x: interval_winner(str(x)))
wd['game_2_interval_winner'] = wd['game_2_interval_score'].apply(lambda x: interval_winner(str(x)))
wd['game_3_interval_winner'] = wd['game_3_interval_score'].apply(lambda x: interval_winner(str(x)))

In [153]:
# Similar to feature creation before, the new column 'game_3_interval_score' will contain erroneous '2's
# As we are simply applying a basic function. Replace these with '0' where appropriate
wd['game_3_interval_score'] = wd['game_3_interval_score'].fillna('0-0')
wd.loc[wd.game_3_interval_score=='0-0', 'game_3_interval_winner']=0

In [154]:
wd.drop(columns=['tournament',
                     'city',
                     'country',
                     'date',
                     'tournament_type',
                     'round',
                     'team_one_player_one',
                     'team_one_player_two',
                     'team_two_player_one',
                     'team_two_player_two',
                     'team_one_player_one_nationality',
                     'team_one_player_two_nationality',
                     'team_two_player_one_nationality',
                     'team_two_player_two_nationality',
                     'team_one_total_points',
                     'team_two_total_points',], inplace=True)

In [189]:
wd.to_csv('data_cleaned/wd.csv', index=False)

#### Men's Doubles (MD)

In [156]:
md['game_2_score'] = md['game_2_score'].fillna('0-0')
md['game_3_score'] = md['game_3_score'].fillna('0-0')
md['team_one_most_consecutive_points_game_2'] = md['team_one_most_consecutive_points_game_2'].fillna(0)
md['team_two_most_consecutive_points_game_2'] = md['team_two_most_consecutive_points_game_2'].fillna(0)
md['team_one_most_consecutive_points_game_3'] = md['team_one_most_consecutive_points_game_3'].fillna(0)
md['team_two_most_consecutive_points_game_3'] = md['team_two_most_consecutive_points_game_3'].fillna(0)
md['team_one_game_points_game_2'] = md['team_one_game_points_game_2'].fillna(0)
md['team_two_game_points_game_2'] = md['team_two_game_points_game_2'].fillna(0)
md['team_one_game_points_game_3'] = md['team_one_game_points_game_3'].fillna(0)
md['team_two_game_points_game_3'] = md['team_two_game_points_game_3'].fillna(0)

In [157]:
md = md[md.retired ==  False]
md.drop(columns=['retired'], inplace=True)

In [158]:
# Create two new columns containing each player's points in game one by splitting 'game_1_score'
md[['team_one_game_1_points', 'team_two_game_1_points']] = md.game_1_score.str.split('-', expand=True)
md[['team_one_game_2_points', 'team_two_game_2_points']] = md.game_2_score.str.split('-', expand=True)
md[['team_one_game_3_points', 'team_two_game_3_points']] = md.game_3_score.str.split('-', expand=True)

In [159]:
# Replace Null values, which we identified do not occur in any game one columns, with zero
md['team_one_game_2_points'] = md['team_one_game_2_points'].fillna(0)
md['team_two_game_2_points'] = md['team_two_game_2_points'].fillna(0)
md['team_one_game_3_points'] = md['team_one_game_3_points'].fillna(0)
md['team_two_game_3_points'] = md['team_two_game_3_points'].fillna(0)

In [160]:
# Create new columns based on above with number representing player who won each game
# As game can go beyond 21 condition is simply >
# np.where() returns player one ('1') if condition is met, else returns player two ('2')
md['game_1_winner'] = np.where(md['team_one_game_1_points'] > md['team_two_game_1_points'], 1, 2)
md['game_2_winner'] = np.where(md['team_one_game_2_points'] > md['team_two_game_2_points'], 1, 2)
md['game_3_winner'] = np.where(md['team_one_game_3_points'] > md['team_two_game_3_points'], 1, 2)

In [161]:
# Replace the erroneous '2's with '0's when a game three was not played
md.loc[((wd.team_one_game_3_points==0) & (md.team_two_game_3_points==0)), 'game_3_winner']=0

In [162]:
# Extract the score at the mid-game interval from the List of each games' score.
md['game_1_interval_score'] = md['game_1_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')
md['game_2_interval_score'] = md['game_2_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')
md['game_3_interval_score'] = md['game_3_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')

In [163]:
# Apply above function to dataframe, we can now compare how influential reaching the Game 1 interval is to winning the first game
md['game_1_interval_winner'] = md['game_1_interval_score'].apply(lambda x: interval_winner(str(x)))
md['game_2_interval_winner'] = md['game_2_interval_score'].apply(lambda x: interval_winner(str(x)))
md['game_3_interval_winner'] = md['game_3_interval_score'].apply(lambda x: interval_winner(str(x)))

In [164]:
# Similar to feature creation before, the new column 'game_3_interval_score' will contain erroneous '2's
# As we are simply applying a basic function. Replace these with '0' where appropriate
md['game_3_interval_score'] = md['game_3_interval_score'].fillna('0-0')
md.loc[md.game_3_interval_score=='0-0', 'game_3_interval_winner']=0

In [165]:
md.drop(columns=['tournament',
                     'city',
                     'country',
                     'date',
                     'tournament_type',
                     'round',
                     'team_one_player_one',
                     'team_one_player_two',
                     'team_two_player_one',
                     'team_two_player_two',
                     'team_one_player_one_nationality',
                     'team_one_player_two_nationality',
                     'team_two_player_one_nationality',
                     'team_two_player_two_nationality',
                     'team_one_total_points',
                     'team_two_total_points',], inplace=True)

In [190]:
md.to_csv('data_cleaned/md.csv', index=False)

#### Mixed Doubles (XD)

In [167]:
xd['game_2_score'] = xd['game_2_score'].fillna('0-0')
xd['game_3_score'] = xd['game_3_score'].fillna('0-0')
xd['team_one_most_consecutive_points_game_2'] = xd['team_one_most_consecutive_points_game_2'].fillna(0)
xd['team_two_most_consecutive_points_game_2'] = xd['team_two_most_consecutive_points_game_2'].fillna(0)
xd['team_one_most_consecutive_points_game_3'] = xd['team_one_most_consecutive_points_game_3'].fillna(0)
xd['team_two_most_consecutive_points_game_3'] = xd['team_two_most_consecutive_points_game_3'].fillna(0)
xd['team_one_game_points_game_2'] = xd['team_one_game_points_game_2'].fillna(0)
xd['team_two_game_points_game_2'] = xd['team_two_game_points_game_2'].fillna(0)
xd['team_one_game_points_game_3'] = xd['team_one_game_points_game_3'].fillna(0)
xd['team_two_game_points_game_3'] = xd['team_two_game_points_game_3'].fillna(0)

In [168]:
xd = xd[xd.retired ==  False]
xd.drop(columns=['retired'], inplace=True)

In [169]:
# Create two new columns containing each player's points in game one by splitting 'game_1_score'
xd[['team_one_game_1_points', 'team_two_game_1_points']] = xd.game_1_score.str.split('-', expand=True)
xd[['team_one_game_2_points', 'team_two_game_2_points']] = xd.game_2_score.str.split('-', expand=True)
xd[['team_one_game_3_points', 'team_two_game_3_points']] = xd.game_3_score.str.split('-', expand=True)

In [170]:
# Replace Null values, which we identified do not occur in any game one columns, with zero
xd['team_one_game_2_points'] = xd['team_one_game_2_points'].fillna(0)
xd['team_two_game_2_points'] = xd['team_two_game_2_points'].fillna(0)
xd['team_one_game_3_points'] = xd['team_one_game_3_points'].fillna(0)
xd['team_two_game_3_points'] = xd['team_two_game_3_points'].fillna(0)

In [171]:
# Create new columns based on above with number representing player who won each game
# As game can go beyond 21 condition is simply >
# np.where() returns player one ('1') if condition is met, else returns player two ('2')
xd['game_1_winner'] = np.where(xd['team_one_game_1_points'] > xd['team_two_game_1_points'], 1, 2)
xd['game_2_winner'] = np.where(xd['team_one_game_2_points'] > xd['team_two_game_2_points'], 1, 2)
xd['game_3_winner'] = np.where(xd['team_one_game_3_points'] > xd['team_two_game_3_points'], 1, 2)

In [172]:
# Replace the erroneous '2's with '0's when a game three was not played
xd.loc[((xd.team_one_game_3_points==0) & (xd.team_two_game_3_points==0)), 'game_3_winner']=0

In [173]:
# Extract the score at the mid-game interval from the List of each games' score.
xd['game_1_interval_score'] = xd['game_1_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')
xd['game_2_interval_score'] = xd['game_2_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')
xd['game_3_interval_score'] = xd['game_3_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')

In [174]:
# Apply above function to dataframe, we can now compare how influential reaching the Game 1 interval is to winning the first game
xd['game_1_interval_winner'] = xd['game_1_interval_score'].apply(lambda x: interval_winner(str(x)))
xd['game_2_interval_winner'] = xd['game_2_interval_score'].apply(lambda x: interval_winner(str(x)))
xd['game_3_interval_winner'] = xd['game_3_interval_score'].apply(lambda x: interval_winner(str(x)))

In [175]:
# Similar to feature creation before, the new column 'game_3_interval_score' will contain erroneous '2's
# As we are simply applying a basic function. Replace these with '0' where appropriate
xd['game_3_interval_score'] = xd['game_3_interval_score'].fillna('0-0')
xd.loc[xd.game_3_interval_score=='0-0', 'game_3_interval_winner']=0

In [176]:
xd.drop(columns=['tournament',
                     'city',
                     'country',
                     'date',
                     'tournament_type',
                     'round',
                     'team_one_player_one',
                     'team_one_player_two',
                     'team_two_player_one',
                     'team_two_player_two',
                     'team_one_player_one_nationality',
                     'team_one_player_two_nationality',
                     'team_two_player_one_nationality',
                     'team_two_player_two_nationality',
                     'team_one_total_points',
                     'team_two_total_points',], inplace=True)

In [191]:
xd.to_csv('data_cleaned/xd.csv', index=False)

#### Women's Singles (WS)

In [178]:
ws['game_2_score'] = ws['game_2_score'].fillna('0-0')
ws['game_3_score'] = ws['game_3_score'].fillna('0-0')
ws['team_one_most_consecutive_points_game_2'] = ws['team_one_most_consecutive_points_game_2'].fillna(0)
ws['team_two_most_consecutive_points_game_2'] = ws['team_two_most_consecutive_points_game_2'].fillna(0)
ws['team_one_most_consecutive_points_game_3'] = ws['team_one_most_consecutive_points_game_3'].fillna(0)
ws['team_two_most_consecutive_points_game_3'] = ws['team_two_most_consecutive_points_game_3'].fillna(0)
ws['team_one_game_points_game_2'] = ws['team_one_game_points_game_2'].fillna(0)
ws['team_two_game_points_game_2'] = ws['team_two_game_points_game_2'].fillna(0)
ws['team_one_game_points_game_3'] = ws['team_one_game_points_game_3'].fillna(0)
ws['team_two_game_points_game_3'] = ws['team_two_game_points_game_3'].fillna(0)

In [179]:
ws = ws[ws.retired ==  False]
ws.drop(columns=['retired'], inplace=True)

In [180]:
# Create two new columns containing each player's points in game one by splitting 'game_1_score'
ws[['team_one_game_1_points', 'team_two_game_1_points']] = ws.game_1_score.str.split('-', expand=True)
ws[['team_one_game_2_points', 'team_two_game_2_points']] = ws.game_2_score.str.split('-', expand=True)
ws[['team_one_game_3_points', 'team_two_game_3_points']] = ws.game_3_score.str.split('-', expand=True)

In [181]:
# Replace Null values, which we identified do not occur in any game one columns, with zero
ws['team_one_game_2_points'] = ws['team_one_game_2_points'].fillna(0)
ws['team_two_game_2_points'] = ws['team_two_game_2_points'].fillna(0)
ws['team_one_game_3_points'] = ws['team_one_game_3_points'].fillna(0)
ws['team_two_game_3_points'] = ws['team_two_game_3_points'].fillna(0)

In [182]:
# Create new columns based on above with number representing player who won each game
# As game can go beyond 21 condition is simply >
# np.where() returns player one ('1') if condition is met, else returns player two ('2')
ws['game_1_winner'] = np.where(ws['team_one_game_1_points'] > ws['team_two_game_1_points'], 1, 2)
ws['game_2_winner'] = np.where(ws['team_one_game_2_points'] > ws['team_two_game_2_points'], 1, 2)
ws['game_3_winner'] = np.where(ws['team_one_game_3_points'] > ws['team_two_game_3_points'], 1, 2)

In [183]:
# Replace the erroneous '2's with '0's when a game three was not played
ws.loc[((ws.team_one_game_3_points==0) & (ws.team_two_game_3_points==0)), 'game_3_winner']=0

In [184]:
# Extract the score at the mid-game interval from the List of each games' score.
ws['game_1_interval_score'] = ws['game_1_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')
ws['game_2_interval_score'] = ws['game_2_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')
ws['game_3_interval_score'] = ws['game_3_scores'].apply(lambda x: interval_score(x.split(',')) if isinstance(x, str) else '0-0')

In [185]:
# Apply above function to dataframe, we can now compare how influential reaching the Game 1 interval is to winning the first game
ws['game_1_interval_winner'] = ws['game_1_interval_score'].apply(lambda x: interval_winner(str(x)))
ws['game_2_interval_winner'] = ws['game_2_interval_score'].apply(lambda x: interval_winner(str(x)))
ws['game_3_interval_winner'] = ws['game_3_interval_score'].apply(lambda x: interval_winner(str(x)))

In [186]:
# Similar to feature creation before, the new column 'game_3_interval_score' will contain erroneous '2's
# As we are simply applying a basic function. Replace these with '0' where appropriate
ws['game_3_interval_score'] = ws['game_3_interval_score'].fillna('0-0')
ws.loc[ws.game_3_interval_score=='0-0', 'game_3_interval_winner']=0

In [187]:
ws.drop(columns=['tournament',
                     'city',
                     'country',
                     'date',
                     'tournament_type',
                     'round',
                     'team_one_nationalities',
                     'team_two_nationalities',
                     'team_one_total_points',
                     'team_two_total_points',], inplace=True)

In [192]:
ws.to_csv('data_cleaned/ws.csv', index=False)